## Run Object Detection model (for Deployment)

Run in the Jetbot with this progam and the trained model

This part of the program is to configure the Jetbet camera and  start the webcam

In [1]:
# Creating live camera display

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# camera = Camera.instance(width=224, height=224, fps=2)
# print(dir(camera))
from jetbot.camera.zmq_camera import ZmqCamera
camera = ZmqCamera(width=224, height=224)  # this should have same interface as current camera

# print(dir(camera))

image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image]))

### TFOD API imports
Here are the imports of the required object detection modules in TFOD API

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

To start software to control the Robot hardware

In [4]:
from jetbot import Robot

robot = Robot()
robot.stop()

## Load and run a model in Python on Linux platform
Start the Tensorflow lite Interpreter to load the trained weight.

Get the intrepreter Input - to input image for prediction

Get the intrepreter Output - to get the predicted ouput

In [5]:
# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as viz_utils
# import matplotlib.pyplot as plt

def detect_object(image_path):
    test_image_np = np.expand_dims(load_image_into_numpy_array(image_path, 300), axis=0)

    input_tensor = tf.convert_to_tensor(test_image_np, dtype=tf.float32)


    # preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    preprocessed_image = tf.keras.applications.mobilenet_v2.preprocess_input(input_tensor)

    interpreter.set_tensor(input_details[0]['index'], preprocessed_image.numpy())

    interpreter.invoke()

    det_boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    det_classes = interpreter.get_tensor(output_details[1]['index'])[0]
    det_scores = interpreter.get_tensor(output_details[2]['index'])[0]
    num_det = interpreter.get_tensor(output_details[3]['index'])[0]    

    print("det_boxes:", det_boxes)
    print("det_classes:", det_classes)
    print("det_scores:", det_scores)
    print("num_det:", num_det)
    
    boundbox = {"xmin": int(det_boxes[0][1] * 224),
                "ymin": int(det_boxes[0][0] * 224),
                "xmax": int(det_boxes[0][3] * 224),
                "ymax": int(det_boxes[0][2] * 224)
               }
    
    return int(det_classes[0]), boundbox

In [6]:
# Creating function to interprete the image and output predictions

import numpy as np
import tensorflow as tf
import time
import PIL
import io

category = ["straight", "right_corner", "approach_corner", "exit_right", "exit_completed"]
position_region = ["ext_left", "left", "middle", "right", "ext_right"]
xpos_region = [0, 40, 85, 139, 184, 224]  # Width 40, 45, 56, 45, 40

# prediction = ""

interpreter = tf.lite.Interpreter(model_path=r"tflite/trained_model04_float16_q.tflite") # result ok but slow
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()    


print(input_details)
print()
print(output_details)



    

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1, 300, 300,   3], dtype=int32), 'shape_signature': array([  1, 300, 300,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

[{'name': 'StatefulPartitionedCall:3', 'index': 247, 'shape': array([ 1, 10,  4], dtype=int32), 'shape_signature': array([ 1, 10,  4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:2', 'index': 248, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype

In [7]:
def analyse_boundbox(img, boundbox):
    rect_length = 5
    x1 = boundbox['xmin']
    x2 = boundbox['xmax']
    y1 = boundbox['ymin']
    y2 = boundbox['ymax']
    
    box = img[y1:y2, x1:x2]
    box_img = Image.fromarray(box, 'L')
    box_img.show()
    # print(x1, x2, y1, y2)
    # print(box.shape)
    
    box_width, box_length = box.shape
    strip = box[box_width - (50 + rect_length) : box_width-50, :]  # Top strip
    
    filter_topstrip = filter_strip(strip)
#     print("\nfilter_topstrip:", filter_topstrip)
#     filter_top_img = Image.fromarray(filter_topstrip, 'L')
#     filter_top_img.show()
    top_position = findposition(filter_topstrip) + x1

    
    box_width, box_length  = box.shape
    strip = box[box_width - (10 + rect_length) : box_width-10, :]  # Bottom strip
    
    filter_bottomstrip = filter_strip(strip)
#     print("\nfilter_bottomstrip:", filter_bottomstrip)
#     strip_bottom_img = Image.fromarray(filter_bottomstrip, 'L')
#     strip_bottom_img.show()
    position = findposition(filter_bottomstrip) + x1
    
    if (top_position != position):
        gradient = (50 - 10) / (top_position - position)
    else:
        gradient = 9999
    
    return position, gradient

In [8]:
def dilution(strip):
    rect_length = 5
    strip_height, strip_width = strip.shape
    
    dilute_strip = np.zeros((strip_height, strip_width - rect_length)).astype(np.uint8)
    
    for h in range(strip_height):
        for w in range(strip_width - rect_length):
            dilute_strip[h, w] = strip[h, w:(w + rect_length)].max()
            
#     print("dilute_strip", dilute_strip)
    return dilute_strip    

In [9]:
def erosion(strip):
    rect_length = 5
    strip_height, strip_width = strip.shape
    
    erode_strip = np.ones((strip_height, strip_width)).astype(np.uint8)
    erode_strip *= np.uint8(255)
    
    for h in range(strip_height):
        for w in range(strip_width - rect_length):
            erode_strip[h, w] = strip[h, w:(w + rect_length)].min()
    
#     print("erode_strip", erode_strip)
    return erode_strip    

In [10]:
def filter_strip(strip):
    rect_length = 5
    bwstrip = bwImg(strip)
    erode_strip = erosion(bwstrip)
    filter_strip = dilution(erode_strip)
    return filter_strip 

In [11]:
def bwImg(img):
    height, width = img.shape
    for h in range(height):
        for w in range(width):
            img[h, w] = (255  if img[h, w] > 90 else 0)
    return img

In [12]:
def findposition(img):
    height, width = img.shape
    for h in range(height):
        for w in range(width):
            if img[h, w] == 0:
                return w
    return width

In [13]:
def move_left(type):
    if type == "gentle":
        print("move_left_gentle")
        left = 0.2
        right = 0.3
        sleep_time = 0.2
    elif type == "steep":
        print("move_left_steep")
        left = 0.1
        right = 0.3
        sleep_time = 0.25
    robot.set_motors(left, right)
    time.sleep(sleep_time)
    robot.stop()

In [14]:
def forward(type):
    if type == "gentle":
        # print("forward_gentle")
        left = 0.4
        right = 0.4
        sleep_time = 0.3
    elif type == "turn_gentle":
        # print("move_right_gentle")
        left = 0.3
        right = 0.3
        sleep_time = 0.2
    elif type == "steep":
        # print("forward_steep")
        left = 0.5
        right = 0.5
        sleep_time = 0.4
    robot.set_motors(left, right)
    time.sleep(sleep_time)
    robot.stop()

In [15]:
def move_right(type):
    if type == "gentle":
        # print("move_right_gentle")
        left = 0.3
        right = 0.2
        sleep_time = 0.2
    elif type == "steep":
        # print("move_right_steep")
        left = 0.3
        right = 0.1
        sleep_time = 0.25
    robot.set_motors(left, right)
    time.sleep(sleep_time)
    robot.stop()

In [16]:
def road_following_straight(boundbox, position, gradient):
    
    if (position >= xpos_region[5 -1]):  # In region 5
        # print("straight - region 5")
        move_right("steep")
        
    elif (position >= xpos_region[4 -1]):  # In region 4
        # print("straight - region 4")
        move_right("steep")
        
    elif (position >= xpos_region[3 - 1]):  # In region 3
        # print("straight - region 3")
        forward("gentle")
        
    elif (position >= xpos_region[2 - 1]):  # In region 2
        # print("straight - region 2")
        move_left("gentle")
        
    elif (position >= xpos_region[1 - 1]):  # In region 1
        # print("straight - region 1")
        move_left("steep")
    

In [17]:
def road_following_approach_corner(boundbox, position, gradient):
    x1 = boundbox['xmin']
    x2 = boundbox['xmax']
    y1 = boundbox['ymin']
    y2 = boundbox['ymax']
    
    if (y2 - y1) > 140:  # Distance away from corner is still quite far, continue with current direction
        # print("approach_corner - straight")
        road_following_straight(boundbox, position, gradient)
    else:
        if (position >= xpos_region[5 -1]):  # In region 5
            # print("approach_corner - region 5")
            move_right("steep")
        elif (position >= xpos_region[4 -1]):  # In region 4
            # print("approach_corner - region 4")
            move_right("gentle")
        elif (position < xpos_region[1]):  # In region 1
            # print("approach_corner - region 1")
            move_left("gentle")
        else:
            # print("approach_corner - region 2 || 3")
            forward("gentle")



In [18]:
def road_following_right_corner(boundbox, position, gradient):
    x1 = boundbox['xmin']
    x2 = boundbox['xmax']
    y1 = boundbox['ymin']
    y2 = boundbox['ymax']
    
    if (y2 - y1) > 100:  # Distance away from corner is still quite far, continue with current direction
        if (position >= xpos_region[5 -1]):  # In region 5
            # print("right_corner - right, region 5, far")
            move_right("gentle")
        else:
            # print("right_corner - forward, region 1,2,3,4, far")
            forward("turn_gentle")        
    else:
        if (position >= xpos_region[5 -1]):  # In region 5
            # print("right_corner - right steep, region 5, near")
            move_right("gentle")
        else:
            # print("right_corner - right gentle, region 1,2,3,4, near")
            move_right("gentle")

In [19]:
def road_following_exit_corner(boundbox, position, gradient):
    x1 = boundbox['xmin']
    x2 = boundbox['xmax']
    y1 = boundbox['ymin']
    y2 = boundbox['ymax']
    
    move_right("gentle")

In [20]:
def road_following_exit_complete(boundbox, position, gradient):
    x1 = boundbox['xmin']
    x2 = boundbox['xmax']
    y1 = boundbox['ymin']
    y2 = boundbox['ymax']
    
    if (0 < gradient < 1):  # Track less than 45 degree 
        move_right("steep")
    elif (1 <= gradient < 1.732):  # Track less than 60 degree
        move_right("gentle")
    elif (gradient >= 1.732) or (gradient <= -1.732):  # Track less than 60 degree
        forward("turn_gentle")
    elif (gradient > -1.732):  # Track less than 45 degree 
        move_left("steep")

In [21]:
from PIL import Image

from six import BytesIO

def load_image_into_numpy_array(image, img_size):
    """Load an image from file into a numpy array.

      Puts image into numpy array to feed into tensorflow graph.
      Note that by convention we put it into a numpy array with shape
      (height, width, channels), where channels=3 for RGB.

      Args:
        path: a file path.

      Returns:
        uint8 numpy array with shape (img_height, img_width, 3)
      """
    
    image = image.resize((img_size, img_size), resample =Image.NEAREST)
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [22]:
def update(change):
    global run
    run += 1
    print(run)
    
    img = PIL.Image.open(io.BytesIO(image.value))
    test_image_np = np.expand_dims(load_image_into_numpy_array(img, 300), axis=0)
    
    input_tensor = tf.convert_to_tensor(test_image_np, dtype=tf.float32)
    
    # preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    preprocessed_image = tf.keras.applications.mobilenet_v2.preprocess_input(input_tensor)
    
    interpreter.set_tensor(input_details[0]['index'], preprocessed_image.numpy())
    
    interpreter.invoke()
    
    det_boxes = interpreter.get_tensor(output_details[0]['index'])[0]
    det_classes = interpreter.get_tensor(output_details[1]['index'])[0]
    det_scores = interpreter.get_tensor(output_details[2]['index'])[0]
    num_det = interpreter.get_tensor(output_details[3]['index'])[0]    
    
    print("det_boxes:", det_boxes)
    print("det_classes:", det_classes)
    print("det_scores:", det_scores)
    print("num_det:", num_det)
    
    detclass = int(det_classes[0]) 
    
    boundbox = {"xmin": int(det_boxes[0][1] * 224),
                "ymin": int(det_boxes[0][0] * 224),
                "xmax": int(det_boxes[0][3] * 224),
                "ymax": int(det_boxes[0][2] * 224)
               }
    
    print("boundbox:", boundbox)
        
    position = ["ext_left", "left", "middle", "right", "ext_right"]
    tilt = ["left", "centre", "right"]
    
    img = img.convert('L').resize((224, 224)) #.resize((224, 224)) # .convert('L') changes image into grayscale
    im = np.array(img)  # datatype np.uint8
    
    position, gradient = analyse_boundbox(im, boundbox)
    print("det category:", category[detclass])
    print("position, gradient:", position, gradient)
    
    if detclass == 0:
        road_following_straight(boundbox, position, gradient)
    elif detclass == 2:
        road_following_approach_corner(boundbox, position, gradient)    
            
    elif detclass == 1:
        road_following_right_corner(boundbox, position, gradient)      
        
    elif detclass == 3:
        road_following_exit_corner(boundbox, position, gradient)        
        
    elif detclass == 4:
        road_following_exit_complete(boundbox, position, gradient)        
    
    time.sleep(0.001)

In [23]:
run = 0

In [24]:
update({'new': camera.value})  # we call the function once to intialize
time.sleep(1)
robot.stop()

1
det_boxes: [[ 0.45314077  0.4463539   0.9850702   0.57925177]
 [ 0.32954425  0.41101336  0.7936079   0.9957628 ]
 [ 0.6101029   0.4404052   0.992048    0.5561105 ]
 [ 0.47902757  0.45434093  0.9738862   0.9136896 ]
 [ 0.5324979   0.43994465  1.0213135   0.63412094]
 [ 0.51658595  0.38860244  1.0234619   0.5354213 ]
 [ 0.43753907  0.66953814  0.9888488   0.8620359 ]
 [-0.03298584  0.39996666  0.36807254  0.97537905]
 [ 0.45992476  0.1640121   0.98866135  0.36033526]
 [ 0.40646088  0.38291618  0.95489824  0.6310484 ]]
det_classes: [0. 2. 0. 2. 0. 0. 0. 1. 0. 0.]
det_scores: [0.7489418  0.5833617  0.30760068 0.23646829 0.1957758  0.19183841
 0.18920195 0.14566982 0.13008189 0.11415204]
num_det: 10.0
boundbox: {'xmin': 99, 'ymin': 101, 'xmax': 129, 'ymax': 220}
det category: straight
position, gradient: 99 10.0
straight - region 3
forward_gentle


In [25]:
camera.observe(update, names='value') # this attaches the 'update' function to the 'value' traitlet of our camera

2
det_boxes: [[ 0.44828916  0.44917312  0.9898056   0.5777123 ]
 [ 0.33263272  0.41266882  0.7914743   0.99478614]
 [ 0.60426223  0.45166758  0.9824176   0.56183654]
 [ 0.52561814  0.4410302   1.0225422   0.6278866 ]
 [ 0.4838277   0.46332744  0.97492766  0.9125793 ]
 [ 0.44112375  0.6719646   0.98895967  0.8653045 ]
 [ 0.51934814  0.38712153  1.0241647   0.53583336]
 [ 0.62609434  0.49021068  0.9780381   0.5928515 ]
 [ 0.46919668  0.14568782  0.97058237  0.3877644 ]
 [-0.03488414  0.39600116  0.37002993  0.9761304 ]]
det_classes: [0. 2. 0. 0. 2. 0. 0. 0. 0. 1.]
det_scores: [0.7986945  0.5669529  0.25990146 0.22617933 0.2057817  0.20556623
 0.15206769 0.14355862 0.12111825 0.11560223]
num_det: 10.0
boundbox: {'xmin': 100, 'ymin': 100, 'xmax': 129, 'ymax': 221}
det category: straight
position, gradient: 100 10.0
straight - region 3
forward_gentle
3
det_boxes: [[ 0.4519801   0.45002335  0.9873234   0.57962304]
 [ 0.33207518  0.41297415  0.79348475  0.9963161 ]
 [ 0.47839886  0.45887125  

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetbot-0.4.3-py3.6.egg/jetbot/camera/zmq_camera.py", line 45, in _run
    self.value = image
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 588, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 577, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1210, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1215, in notify_change
    return self._notify_observers(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 125

In [28]:
camera.unobserve_all()


In [29]:
robot.stop()


In [ ]:
camera_link.unlink()